# AVITO PROJECT

## Team Member

* Sandikha Rahardi - РИМ 130908

## Load Data

### Import Package

In [596]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split

### Set Dataframe

In [597]:
dataset_filename = "assets/avito_dataset.xlsx"
main_df = pd.read_excel(dataset_filename, sheet_name="Sheet1")

## Data Cleaning

### Important Column

In [598]:
# Define important column to predict the price
important_columns = [
    'Цена', 'Метро', 'Метро2', 'Метро. Время', 
    'Район', 'Сегодня просмотров', 'Всего просмотров', 
    'Продавец. Компания', 'Статус', 'Количество комнат',
    'Общая площадь, число', 'Этаж', 'Жилая площадь, число',
    'Санузел', 'Балкон или лоджия', 'Окна', 'Ремонт', 
    'Способ продажи', 'Мебель', 'Вид сделки', 'Техника',
    'Тип дома', 'Этажей в доме', 'Год постройки',
    'Пассажирский лифт', 'Парковка', 'Двор', 'Грузовой лифт', 'В доме',
    'Тёплый пол', 'Дополнительно', 'Отделка', 'Название новостройки',
    'Корпус, строение', 'Тип участия', 'Площадь кухни, число', 'Тип комнат'
]
print("Total original column : " , main_df.columns.__len__())
print("Total important column : " , important_columns.__len__())
main_df = main_df[important_columns]

Total original column :  71
Total important column :  37


### Clean the dataset

#### Clean about Метро

In [599]:
main_df['Метро'].dropna(inplace=True)

def preprocess_metro_time(row):
    if(type(row) == str):
        row = row.replace("мин.","").strip()
        if(row.__contains__("от")):
            row = float(row.replace("от","").strip())
        elif(row.__contains__("до")):
            row = float(row.replace("до","").strip())
        elif(row.__contains__("–")):
            row = (int(row.split("–")[0]) + int(row.split("–")[1])) / 2
    return row

main_df['Метро. Время'] = main_df['Метро. Время'].apply(preprocess_metro_time)
mean_metro_time = round(pd.to_numeric(main_df['Метро. Время']).mean(), 1)
main_df['Метро. Время'].fillna(mean_metro_time, inplace=True)

main_df['Метро2'] = main_df['Метро2'].apply(lambda x: "Нет" if pd.isna(x) else "Есть")

#### Clean about 'Район'

In [600]:
main_df['Район'].fillna("Другой", inplace=True)

#### Clean about 'Просмотров'

In [601]:
main_df['Сегодня просмотров'].fillna(0, inplace=True)
main_df['Всего просмотров'].fillna(0, inplace=True)

#### Clean about 'Компания'

In [602]:
main_df['Продавец. Компания'].dropna(inplace=True)

#### Clean about 'Статус'

In [603]:
main_df['Статус'].dropna(inplace=True)

#### Clean about 'Количество комнат'

In [604]:
main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: 0 if x == 'Студия' else x)

max_room = pd.to_numeric(main_df['Количество комнат'], errors='coerce').max()
mean_room = pd.to_numeric(main_df['Количество комнат'], errors='coerce').mean()

main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: max_room if x == 'многокомнатные' else x)
main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: round(mean_room) if x == 'Своб. планировка' else x)

main_df['Количество комнат'].value_counts()

Количество комнат
2.0    6574
1.0    6343
3.0    3626
0.0    2682
4.0     494
5.0      67
6.0       9
7.0       8
Name: count, dtype: int64

> Note: Студия does not have room ( 0 room )

#### Clean about 'Общая площадь, число'

In [605]:
main_df['Общая площадь, число'].dropna(inplace=True)

main_df['Общая площадь, число'] = main_df['Общая площадь, число'].apply(lambda x: float(x.replace(",", ".")) if type(x) == str else int(x))

#### Clean about 'Этаж'

In [606]:
# Clean dataset 'Этаж' of 'Этажей в доме'
main_df['Этаж'] = main_df['Этаж'].apply(lambda x: int(x.split("из")[0].strip()))

### Clean about 'Жилая площадь, число'

In [607]:
main_df['Жилая площадь, число'].fillna(0, inplace=True)

#### Clean about 'Санузел'

In [608]:
main_df['Санузел'].fillna('Другой', inplace=True)

#### Clean about 'Балкон или лоджия'

In [609]:
main_df['Балкон или лоджия'].fillna('Другой', inplace=True)

#### Clean about 'Окна'

In [610]:
main_df['Окна'].fillna('Другой', inplace=True)

#### Clean about 'Ремонт'

In [611]:
main_df['Ремонт'].fillna('Другой', inplace=True)

#### Clean about 'Способ продажи'

In [612]:
main_df['Способ продажи'].fillna('Другой', inplace=True)

#### Clean about 'Мебель'

In [613]:
main_df['Мебель'].fillna('Другой', inplace=True)

#### Clean about 'Вид сделки'

In [614]:
main_df['Вид сделки'].fillna('Другой', inplace=True)

#### Clean about 'Техника'

In [615]:
main_df['Техника'].fillna(0, inplace=True)

def preprocess_tech(row):
    if(type(row) == str):
        if(row.__contains__(",")):
            row = len(row.split(","))
        else: 
            row = 1
    return row
main_df['Техника'] = main_df['Техника'].apply(preprocess_tech)

#### Clean about 'Тип дома'

In [616]:
main_df['Тип дома'].fillna('Другой', inplace=True)

#### Clean about 'Год постройки'

In [617]:
# Change null value with median
print("Data Null :" , main_df['Год постройки'].isna().sum())
print("Data Mode :" , main_df['Год постройки'].mode())
main_df['Год постройки'].fillna(main_df['Год постройки'].median(), inplace=True)

Data Null : 13832
Data Mode : 0    2023.0
Name: Год постройки, dtype: float64


#### Clean about 'Пассажирский лифт'

In [618]:
# Change null value with mode
main_df['Пассажирский лифт'] = main_df['Пассажирский лифт'].apply(lambda x: 0 if x == 'Нет' else x)
print("Data Null :" , main_df['Пассажирский лифт'].isna().sum())
print("Data Mode :" , main_df['Пассажирский лифт'].mode())
main_df['Пассажирский лифт'].fillna(main_df['Пассажирский лифт'].mode(), inplace=True)

Data Null : 11917
Data Mode : 0    1.0
Name: Пассажирский лифт, dtype: float64


#### Clean about 'Парковка'

In [619]:
main_df['Парковка'].fillna(0, inplace=True)

def preprocess_parking(row):
    if(type(row) == str):
        if(row.__contains__(",")):
            row = len(row.split(","))
        else: 
            row = 1
    return row
main_df['Парковка'] = main_df['Парковка'].apply(preprocess_parking)

### Clean about 'Двор'

In [620]:
main_df['Двор'].fillna(0, inplace=True)

def preprocess_parking(row):
    if(type(row) == str):
        if(row.__contains__(",")):
            row = len(row.split(","))
        else: 
            row = 1
    return row
main_df['Двор'] = main_df['Двор'].apply(preprocess_parking)

### Clean about 'Грузовой лифт'

In [621]:
# Change null value with mode
main_df['Грузовой лифт'] = main_df['Грузовой лифт'].apply(lambda x: 0 if x == 'Нет' else x)
print("Data Null :" , main_df['Грузовой лифт'].isna().sum())
print("Data Mode :" , main_df['Грузовой лифт'].mode())
main_df['Грузовой лифт'].fillna(main_df['Грузовой лифт'].mode(), inplace=True)

Data Null : 13208
Data Mode : 0    1.0
Name: Грузовой лифт, dtype: float64


### Split Dataset

In [87]:
# Split the data into training (80%), testing (15%), and validation (5%)
train_df, temp_df = train_test_split(main_df, test_size=0.2, random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=0.25, random_state=42)

In [88]:
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

(15842, 38)
(2970, 38)
(991, 38)


### Devided into Categorical and Numerical dataset

In [89]:
numerical_columns = ['Цена', 'Метро. Время', 'Сегодня просмотров', 'Всего просмотров', 'Количество комнат', 'Общая площадь, число', 'Этаж', 'Жилая площадь, число', 'Этажей в доме', 'Год постройки', 'Пассажирский лифт', 'Грузовой лифт', 'Площадь кухни, число', 'Техника']
categorical_columns = ['Метро', 'Метро2', 'Район', 'Продавец. Компания', 'Статус', 'Санузел', 'Балкон или лоджия', 'Окна', 'Ремонт', 'Способ продажи', 'Мебель', 'Вид сделки', 'Тип дома', 'Парковка', 'Двор', 'В доме', 'Тёплый пол', 'Дополнительно', 'Отделка', 'Название новостройки', 'Корпус, строение', 'Тип участия', 'Тип комнат']

train_num_df = train_df[numerical_columns]
train_cat_df = train_df[categorical_columns]
test_num_df = test_df[numerical_columns]
test_cat_df = test_df[categorical_columns]
val_num_df = val_df[numerical_columns]
val_cat_df = val_df[categorical_columns]